In [1]:
from google.oauth2 import service_account
from google.cloud import vision
import argparse
import cv2
import io

In [2]:
# def draw_ocr_results(image, text, rect, color=(0, 255, 0)):
# 	# unpacking the bounding box rectangle and draw a bounding box
# 	# surrounding the text along with the OCR'd text itself
# 	(startX, startY, endX, endY) = rect
# 	cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
# 	cv2.putText(image, text, (startX, startY - 10),
# 		cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
# 	# return the output image
# 	return image

In [3]:
# create the client interface to access the Google Cloud Vision API
credentials = service_account.Credentials.from_service_account_file(
	filename="app\\google_cloud_creds.json",
	scopes=["https://www.googleapis.com/auth/cloud-platform"])
client = vision.ImageAnnotatorClient(credentials=credentials)
# load the input image as a raw binary file (this file will be
# submitted to the Google Cloud Vision API)
with io.open("wring.jpg", "rb") as f:
	byteImage = f.read()

In [4]:
print("[INFO] making request to Google Cloud Vision API...")
image = vision.Image(content=byteImage)
response = client.text_detection(image=image)
# check to see if there was an error when making a request to the API
if response.error.message:
	raise Exception(
		"{}\nFor more info on errors, check:\n"
		"https://cloud.google.com/apis/design/errors".format(
			response.error.message))

[INFO] making request to Google Cloud Vision API...


In [5]:
# image = cv2.imread("images\\voterid_1.jpg")
# final = image.copy()
# for text in response.text_annotations[1::]:
# 	# grab the OCR'd text and extract the bounding box coordinates of
# 	# the text region
# 	ocr = text.description
# 	startX = text.bounding_poly.vertices[0].x
# 	startY = text.bounding_poly.vertices[0].y
# 	endX = text.bounding_poly.vertices[1].x
# 	endY = text.bounding_poly.vertices[2].y
# 	# construct a bounding box rectangle from the box coordinates
# 	rect = (startX, startY, endX, endY)
# 		# draw the output OCR line-by-line
# 	output = image.copy()
# 	output = draw_ocr_results(output, ocr, rect)
# 	final = draw_ocr_results(final, ocr, rect)
# 	# show the output OCR'd line
# 	print(ocr)
# # show the final output image
# cv2.imshow("Final Output", final)
# cv2.waitKey(0)

In [6]:
ocr_texts = []
for text in response.text_annotations[1::]:
    ocr_texts.append(text.description)

In [7]:
ocr_texts = [ocr.lower() for ocr in ocr_texts]

In [8]:
ocr_texts = " ".join(ocr_texts)

In [9]:
ocr_texts

'in case this card is lost / found , kindly inform income tax pan services unit , utitsl plot no. 3 , sector 11 , cbd belapur , navi mumbai - 400614 . इस कार्ड के खोने / पाने पर कृपया सूचित करें / लौटाएं : आयकर पैन सेवा यूनीट , यूटीआईटीएसएल प्लाट नं : ३ , सेक्टर ११ , सी.बी.डी.बेलापूर , नवी मुंबई -४०० ६१४ . आधार भारतीय विशिष्ट पहचान प्राधिकरण unique identification authority of india पता : संबोधितः कमलदीप सिंह अरोरा , ए / 602 सुन्दर आंगन , बेवरली पार्क , सिनेमेक्स , कनकिया मिरा रोड ईस्ट , ठाणे , ठाणे , महाराष्ट्र - 401107 1947 address : w / o : kamaldeep singh arora , a / 602 sundar angan , beverly park , cinemax , kanakia mira road east , thane , thane , maharashtra - 401107 2117 0180 0601 help@uidai.gov.in www www.uidai.gov.in'

For passport add maybe the "<" keyword for search.

In [10]:
doc_keywords = {"aadhar": ["government of india", "unique identification authority", "uidai", "help@uidai.gov.in"],
                "pan": ["income tax department", "permanent account number", "pan", "income tax pan services unit"],
                "passport": ["nationality", "republic of india", "country code"],
                "voter": ["election commission of india", "epic", "elector", "elector's", "electoral registration officer"]}

In [11]:
def classify_text(text, doc_keywords):
    for key, keywords_list in doc_keywords.items():
        for keyword in keywords_list:
            if keyword in text:
                return key
    return "Unclassified"

In [12]:
def classify_text(text, doc_keywords):
    matching_types = []
    for key, keywords_list in doc_keywords.items():
        for keyword in keywords_list:
            if keyword in text:
                matching_types.append(key)

    if not matching_types:
        return ["Unclassified"]
    
    return list(set(matching_types))

In [13]:
doc_type = classify_text(ocr_texts, doc_keywords)

In [14]:
doc_type

['pan', 'aadhar']

In [15]:
side = {
    "voter": 
    {
        "front": ["election commission of india", "elector"],
        "back": ["electoral registration officer"]
    },
    "aadhar":
    {
        "front": ["government of india"],
        "back": ["unique identification authority"]
    },
    "pan":
    {
        "front": ["income tax department of india"],
        "back": ["income tax pan services unit", "nsdl"]
    },
    "passport":
    {
        "front": ["nationality", "republic of india"],
        "back": [""]
    }
}

In [16]:
doc_type

['pan', 'aadhar']

In [17]:
def classify_document_side(document_type, text, doc_keywords, side):
    classified_side = []
    if document_type not in side or document_type not in doc_keywords:
        classified_side.append("Invalid document type")
        return classified_side

    front_keywords = side[document_type]["front"]
    back_keywords = side[document_type]["back"]

    front_present = any(keyword in text for keyword in front_keywords)
    back_present = any(keyword in text for keyword in back_keywords)

    if front_present and back_present:
        classified_side.append("Front")
        classified_side.append("Back")
        return classified_side
    elif front_present:
        classified_side.append("Front")
        return classified_side
    elif back_present:
        classified_side.append("Back")
        return classified_side
    else:
        classified_side.append("Not identified")
        return classified_side

In [19]:
response = {}
for doc in doc_type:
    doc_side = classify_document_side(doc, ocr_texts, doc_keywords, side)
    try:
        response[doc].extend(doc_side)
    except:
        response[doc] = doc_side
response
    


{'pan': ['Back'], 'aadhar': ['Back']}

: 

In [4]:
from PIL import Image

In [5]:
image = Image.open("pan_front.tiff")

In [6]:
out = image.convert("RGB")
out.save("sample.jpg", "JPEG")

In [1]:
import os

In [8]:
os.listdir("tmp2vfqh3f9")

['out0.jpg',
 'out1.jpg',
 'out2.jpg',
 'out3.jpg',
 'out4.jpg',
 'out5.jpg',
 'out6.jpg',
 'out7.jpg',
 'out8.jpg']